In [52]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import time
import pandas as pd
from modules.cleaning_functions import *
from modules.scrapping_functions import *
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Récupérer données du H

In [53]:
date_time_list = []
date_reference = datetime.strptime('2025-02-15 20:00', "%Y-%m-%d %H:%M")

url_h = 'https://hbcnantes.com/equipe-professionnelle/calendrier-et-resultats/'
page = requests.get(url_h)
if page.status_code == 200 :
    soup = BeautifulSoup(page.content, 'html.parser')
    event_list = soup.find('div', class_="calendar").find('div', class_="container")
    calendar_item = event_list.find_all('div',class_='row calendar__row align-items-center no-gutters mb-2')
    
    for item in calendar_item :
        home_team = item.find('div', class_="calendar__row-teamname").text.strip()
        if home_team == 'HBC Nantes' :
            calendar_tag = item.find('div', class_="d-flex align-items-center").find('div', class_="calendar__row-date")
            datime_match = get_Harena_datetime(calendar_tag)
            if (datime_match > date_reference) and (datime_match.time() != time(0, 0, 0)):
                date_time_list.append(datime_match)
df_H_arena = pd.DataFrame({
    'lieu' : 'Palais des Sports Beaulieu / H Arena',
    'nom' : 'Match handball',
    'longitude' : '-1.536772',
    'latitude' : '47.209525',
    'type' : 'Evènement sportif',
    'heure_debut': [dt.strftime('%H:%M') if dt else None for dt in date_time_list], 
    'heure_fin' :[dt.strftime('%H:%M') if dt else None for dt in date_time_list],
    'date' : [time.date() for time in date_time_list], 
    'datetime' : date_time_list,
})

df_H_arena.head()

,lieu,nom,longitude,latitude,type,heure_debut,heure_fin,date,datetime
0,Palais des Sports Beaulieu / H Arena,Match handball,-1.536772,47.209525,Evènement sportif,20:45,20:45,2025-02-19,2025-02-19 20:45:00
1,Palais des Sports Beaulieu / H Arena,Match handball,-1.536772,47.209525,Evènement sportif,20:30,20:30,2025-02-22,2025-02-22 20:30:00
2,Palais des Sports Beaulieu / H Arena,Match handball,-1.536772,47.209525,Evènement sportif,20:30,20:30,2025-03-01,2025-03-01 20:30:00
3,Palais des Sports Beaulieu / H Arena,Match handball,-1.536772,47.209525,Evènement sportif,20:45,20:45,2025-03-05,2025-03-05 20:45:00
4,Palais des Sports Beaulieu / H Arena,Match handball,-1.536772,47.209525,Evènement sportif,20:00,20:00,2025-03-22,2025-03-22 20:00:00


# Récupérer données du FCNantes

In [76]:
url_fcnantes = 'https://www.fcnantes.com/articles/20242025/calendrier.php'
page_fc = requests.get(url_fcnantes)
if page_fc.status_code == 200 :
    soup_fcnantes = BeautifulSoup(page_fc.content, 'html.parser')
    match_container = soup_fcnantes.find(id="pageContainer")

In [91]:

datime_match_list = []
schedule_list = []
date_list = []

matches = match_container.find_all(class_="match")
for match in matches:
    # Correction d'encodage sur les mois
    date_text = match.find(class_="datecal").text.strip().replace('ao√Ľt', 'août').replace('d√©cembre', 'décembre').replace('f√©vrier', 'février')
    
    # Parse propre avec la bonne classe
    date_obj = datetime.strptime(date_text, "%A %d %B %Y").date()

    # Comparaison avec une date
    if date_obj > datetime.strptime('2025-02-14', "%Y-%m-%d").date():
        stade = match.find(class_="stade").text.upper().strip()
        
        if 'BEAUJOIRE' in stade:
            score = match.find(class_="scorecal").text.strip()
            blue_btns = match.find_all('div', class_="blue-btn")

            if blue_btns:
                for btn in blue_btns:
                    a_tag = btn.find('a', class_="first-link")
                    if a_tag and "Compo" in a_tag.text:
                        heure = fetch_datetime_fcmatch(a_tag['href'])
                        if heure: 
                            date_time_match = datetime.combine(date_obj, heure)
                            datime_match_list.append(date_time_match)

            else:
                if score != '- : -':
                    schedule_list.append(score)
                    date_list.append(date_obj)


Heure : 15, Minutes : 0
Time object : 15:00:00 / Type : <class 'datetime.time'>
Heure : 17, Minutes : 15
Time object : 17:15:00 / Type : <class 'datetime.time'>
Heure : 17, Minutes : 0
Time object : 17:00:00 / Type : <class 'datetime.time'>


# Récupérer données STEREOLUX, Cité des Congrés, Zénith

In [54]:
url_nantes_agenda = 'https://openagenda.com/fr/nantesmetropole?locationUid%5B0%5D=47538434&locationUid%5B1%5D=21547859&locationUid%5B2%5D=95545941&locationUid%5B3%5D=41936666&locationUid%5B4%5D=34310089&locationUid%5B5%5D=57179132&locationUid%5B6%5D=79753849&locationUid%5B7%5D=45334118&timings%5Bgte%5D=2025-02-14T23%3A00%3A00.000Z&timings%5Blte%5D=2025-03-25T22%3A59%3A59.999Z&timings%5Btz%5D=Europe%2FParis'

In [55]:
import time

# Configure le WebDriver en auto-download
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(url_nantes_agenda)

# Scroller pour charger tout le contenu
scroll_pause = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

driver.quit()

In [56]:
event_tag = soup.find('div', class_="oa-1o8sgt3").find('div', class_="oa-11ydnqy")
event_list = event_tag.find_all('article', class_="oa-z3hrkb")

In [69]:
place_list = []
datetime_list = []
name_list = []

for event in event_list :
    info_section = event.find('section')
    date_time_section = info_section.find('div', class_="oa-gg4vpm").find_all('li')
    name = info_section.find('div', class_="oa-1bzcof").text.strip().replace('Complet', '')
    date_time = date_time_section[0].text
    place = date_time_section[1].text
    
    if 'Congrès' in place :
        place = 'Cité des Congrès'
    elif 'STEREOLUX' in place :
        place = 'STEREOLUX'
    elif 'Stade de la Beaujoire Louis Fonteneau, Nantes' in place :
        place = 'Stade de la Beaujoire'
    elif 'Palais des Sports Beaulieu, Nantes' :
        place = 'Palais des Sports Beaulieu / H Arena'
    elif 'Zénith' in place :
        place = 'Zénith Nantes Métropole'
    elif 'Parc des Expositions de la Beaujoire, Nantes' in place :
        place = 'Parc des Expos de la Beaujoire / Exponantes Le Parc'
    
    datetimes = extraire_datetimes(date_time)
    if len(datetimes) > 1 :
        amount_datetimes = len(datetimes)
        [datetime_list.append(datetime_) for datetime_ in datetimes]
        [place_list.append(place) for i in range(len(datetimes))]
        [name_list.append(name) for i in range(len(datetimes))]
        
    else :
        place_list.append(place)
        datetime_list.append(datetimes[0] if datetimes else None)
        name_list.append(name)
        
if len(place_list) == len(datetime_list) == len(name_list) :       
    df_agenda_nantes = pd.DataFrame({
        'lieu' : place_list,
        'nom' : name_list,
        'longitude' : '',
        'latitude' : '',
        'type' : 'Culture',
        'heure_debut': [dt.strftime('%H:%M') if dt else None for dt in datetime_list], 
        'heure_fin' : [dt.strftime('%H:%M') if dt else None for dt in datetime_list],
        'date' : [dt.date() if dt else None for dt in datetime_list], 
        'datetime' : datetime_list
    })
else :
    raise Exception('all list must be the same lenght')

In [70]:
df_agenda_nantes.head()

,lieu,nom,longitude,latitude,type,heure_debut,heure_fin,date,datetime
0,Cité des Congrès,"Les hommes viennent de Mars, les femmes de Vén...",,,Culture,17:30,17:30,2025-02-15,2025-02-15 17:30:00
1,Cité des Congrès,"Les hommes viennent de Mars, les femmes de Vén...",,,Culture,20:30,20:30,2025-02-15,2025-02-15 20:30:00
2,Cité des Congrès,Tout va très bien ! - avec Laurent Ournac et A...,,,Culture,17:00,17:00,2025-02-16,2025-02-16 17:00:00
3,STEREOLUX,The Libertines,,,Culture,18:30,18:30,2025-02-16,2025-02-16 18:30:00
4,Cité des Congrès,Simon Superlapin : Superlapin et la pierre qui...,,,Culture,14:00,14:00,2025-02-23,2025-02-23 14:00:00


# Récupérer les autres évenements sur dataset Nantes

In [ ]:
# Liste des lieux à interroger
lieux = [
    "Cité des Congrès",
    "STEREOLUX",
    "Zénith Nantes Métropole",
    "Stade de la Beaujoire",
    "Parc des Expos de la Beaujoire / Exponantes Le Parc",
    "Palais des Sports Beaulieu / H Arena"
]

# URL de base de l'API
BASE_URL = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_agenda-evenements-nantes-metropole_v2/records"

# Date de début de la recherche
DATE_DEBUT = "2025-02-15"

# Liste pour stocker tous les événements
all_events = []

# Boucle sur chaque lieu
for lieu in lieux:
    params = {
        "where": f"date>'{DATE_DEBUT}' AND lieu='{lieu}'"
        }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        events = data.get('results', [])
        all_events.extend(events)  # On ajoute les événements à la liste globale
    else:
        print(f"Erreur pour {lieu} : {response.status_code} - {response.text}")


# Transformation en DataFrame Pandas
df = pd.DataFrame(all_events)

# Sélection des colonnes
colonnes_finales = ['lieu', 'nom', 'longitude', 'latitude', 'heure_debut', 'heure_fin', 'date']
df_final = df[colonnes_finales]


               lieu                                                nom  \
0  Cité des Congrès              Neko Light Orchestra : Echos d’Hyrule   
1  Cité des Congrès                    Jean-Paul Rouve : L’air de rien   
2  Cité des Congrès                       D'jal : En pleine conscience   
3  Cité des Congrès           Olivier de Benoist : Le droit au bonheur   
4  Cité des Congrès  Aterballetto - Centre Chorégraphique National ...   

   longitude   latitude heure_debut heure_fin        date  
0  -1.543612  47.212767       15:30      None  2025-04-13  
1  -1.543612  47.212767       20:00      None  2025-04-18  
2  -1.543612  47.212767       20:00      None  2025-04-26  
3  -1.543612  47.212767       20:00      None  2025-05-20  
4  -1.543612  47.212767       20:45     22:00  2025-04-03  


In [75]:
df_final

,lieu,nom,longitude,latitude,heure_debut,heure_fin,date
0,Cité des Congrès,Neko Light Orchestra : Echos d’Hyrule,-1.543612,47.212767,15:30,None,2025-04-13
1,Cité des Congrès,Jean-Paul Rouve : L’air de rien,-1.543612,47.212767,20:00,None,2025-04-18
2,Cité des Congrès,D'jal : En pleine conscience,-1.543612,47.212767,20:00,None,2025-04-26
3,Cité des Congrès,Olivier de Benoist : Le droit au bonheur,-1.543612,47.212767,20:00,None,2025-05-20
4,Cité des Congrès,Aterballetto - Centre Chorégraphique National ...,-1.543612,47.212767,20:45,22:00,2025-04-03
5,Cité des Congrès,Star Wars - Saison ONPL,-1.543612,47.212767,20:00,21:10,2025-05-22
6,Cité des Congrès,Conseil métropolitain,-1.543612,47.212767,09:00,None,2025-04-04
7,Cité des Congrès,Magma,-1.543612,47.212767,20:00,None,2025-04-15
8,Cité des Congrès,Jenifer : Juke Box Tour,-1.543612,47.212767,20:00,None,2025-04-23
9,Cité des Congrès,Tom Baldetti – Tome 1,-1.543612,47.212767,20:30,None,2025-04-24
